# Necessities

In [1]:
# ------------------------------------------------------------------
# Imports
# ------------------------------------------------------------------
# Basic data processing libraries
import pandas as pd
import numpy as np
import os
import torch

# Graph data processing libraries
import networkx as nx
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx

# Libraries for (G)NNs
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
import torch.nn as nn
from sklearn.metrics import roc_auc_score, f1_score

# ------------------------------------------------------------------
# Helper functions
# ------------------------------------------------------------------
def show_df_info(df):
    print(df.info())
    print('####### Repeat ####### \n', df.duplicated().any())
    print('####### Count ####### \n', df.nunique())
    print('####### Example ####### \n',df.head())

def label_statics(label_df, label_list):
    print("####### nCount #######")
    for label in label_list:
        print(label_df[label].value_counts())
    print("####### nPercent #######")
    for label in label_list:
        print(label_df[label].value_counts()/label_df.shape[0])

# ------------------------------------------------------------------
# Data stuff
# ------------------------------------------------------------------
base_path = os.getcwd()
input_ali_data_path = base_path

# Load the data files
user_labels_path = os.path.join(input_ali_data_path, "credit.csv")
user_edges_path = os.path.join(input_ali_data_path, "credit_edges.csv")

# Create dataframes to store the information from the .csv files
user_labels = pd.read_csv(user_labels_path)
user_edges = pd.read_csv(user_edges_path)

user_labels.insert(0, 'user_id', user_labels.index)

user_edges = user_edges[user_edges['uid1'].isin(user_labels['user_id']) & user_edges['uid2'].isin(user_labels['user_id'])]
user_labels_train = user_labels
user_labels_train = user_labels_train.drop(columns=['NoDefaultNextMonth'])

# Extract node features from user_labels dataframe
node_features = user_labels_train.iloc[:, 1:]
node_features = torch.tensor(node_features.values, dtype=torch.float)

# Extract edges from user_edges dataframe
edges = user_edges[['uid1', 'uid2']]
edges['uid1'] = edges['uid1'].map(dict(zip(user_labels['user_id'], range(len(user_labels)))))
edges['uid2'] = edges['uid2'].map(dict(zip(user_labels['user_id'], range(len(user_labels)))))

# Convert edges dataframe to tensor
edges_tensor = torch.tensor(edges.values, dtype=torch.long).t().contiguous()

# Create edge_index tensor
edge_index = edges_tensor

# Create torch-geometric data
data = Data(x=node_features, edge_index=edge_index)

num_nodes = node_features.size(0)
num_classes = 2 
num_node_features = data.num_node_features

# Create masks for training, and testing
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)

# 60-20-20 Train and Test data split
num_train = int(num_nodes * 0.6)
num_val = int(num_nodes * 0.8)
train_mask[:num_train] = True
val_mask[num_train:num_val] = True
test_mask[num_val:] = True

data.train_mask = train_mask
data.test_mask = test_mask
data.val_mask = val_mask

# Labels from the data (in this case: Job Classification)
data.y = torch.tensor(user_labels['NoDefaultNextMonth'].values, dtype=torch.long)


# ------------------------------------------------------------------
# Set Device
# ------------------------------------------------------------------

def set_device():
    return torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

# ------------------------------------------------------------------
# Loss
# ------------------------------------------------------------------

def fairness_aware_loss(output, data, sensitive_attr, alpha=0, beta=0, gamma=0, delta=0):
    target = data.y[data.train_mask]
    # standard_loss = F.cross_entropy(output, target)
    standard_loss = F.nll_loss(output, target)

    labels = data.y[train_mask]
    pos_prob = torch.exp(output[:, 1])
    neg_prob = torch.exp(output[:, 0])
    # pos_prob = torch.sigmoid(output[:, 1])
    # neg_prob = 1 - pos_prob
    predictions = output.argmax(dim=1)

    # Statistical Parity Regularization
    sp_reg = torch.abs(pos_prob[sensitive_attr == 1].mean() - pos_prob[sensitive_attr == 0].mean())

    # # Calculating FPR and TPR for each group
    # fpr_group1 = ((predictions == 1) & (labels == 0) & (sensitive_attr == 1)).float().mean()
    # fpr_group0 = ((predictions == 1) & (labels == 0) & (sensitive_attr == 0)).float().mean()
    # tpr_group1 = ((predictions == 1) & (labels == 1) & (sensitive_attr == 1)).float().mean()
    # tpr_group0 = ((predictions == 1) & (labels == 1) & (sensitive_attr == 0)).float().mean()

    # Treatment Equality Regularization
    fp_diff = (neg_prob * (labels == 0) * (sensitive_attr == 1)).float().mean() - \
              (neg_prob * (labels == 0) * (sensitive_attr == 0)).float().mean()
    fn_diff = (pos_prob * (labels == 1) * (sensitive_attr == 1)).float().mean() - \
              (pos_prob * (labels == 1) * (sensitive_attr == 0)).float().mean()
    treatment_reg = torch.abs(fp_diff) + torch.abs(fn_diff)
    # treatment_reg = torch.abs(fn_diff)

    # fn_group_1 = ((predictions == 0) & (labels == 1) & (sensitive_attr == 1)).sum()
    # fp_group_1 = ((predictions == 1) & (labels == 0) & (sensitive_attr == 1)).sum()

    # fn_group_0 = ((predictions == 0) & (labels == 1) & (sensitive_attr == 0)).sum()
    # fp_group_0 = ((predictions == 1) & (labels == 0) & (sensitive_attr == 0)).sum()
    
    # ratio_group_1 = fn_group_1 / fp_group_1 if fp_group_1 != 0 else torch.tensor(float('inf'))
    # ratio_group_0 = fn_group_0 / fp_group_0 if fp_group_0 != 0 else torch.tensor(float('inf'))
    # treatment_reg = torch.abs(ratio_group_1 - ratio_group_0)

    # Equal Opportunity Difference Regularization
    eod_reg = torch.abs((pos_prob * (labels == 1) * (sensitive_attr == 1)).float().mean() - \
                        (pos_prob * (labels == 1) * (sensitive_attr == 0)).float().mean())

    # Overall Accuracy Equality Difference Regularization
    oaed_reg = torch.abs((pos_prob * (sensitive_attr == 1)).float().mean() - \
                         (pos_prob * (sensitive_attr == 0)).float().mean())

    penalty = alpha + beta + gamma + delta
    
    # Combine losses
    combined_loss = (1-penalty)*standard_loss
    + alpha * sp_reg
    + beta * treatment_reg
    + gamma * eod_reg
    + delta * oaed_reg
    
    return combined_loss

# ------------------------------------------------------------------
# Fairness Metrics
# ------------------------------------------------------------------

def calculate_fairness(label, predictions, sens_attr='Gender', balanced=False):
    """
    Calculate various fairness metrics.

    Args:
    label: Actual labels (binary).
    predictions: Model predictions (binary).
    sens_attr: Binary sensitive attribute for fairness evaluation.

    Returns:
    A dictionary containing SPD, EOD, OAED, and TED values.
    """
    if balanced is False:
        labels = torch.tensor(user_labels[label].values, dtype=torch.long)
        sensitive_attribute = torch.tensor(user_labels[sens_attr].values, dtype=torch.long)
    else:
        labels = torch.tensor(filtered_user_labels[label].values, dtype=torch.long)
        sensitive_attribute = torch.tensor(filtered_user_labels[sens_attr].values, dtype=torch.long)
    
    labels = labels.to(set_device())
    sensitive_attribute = sensitive_attribute.to(set_device())

    predictions = predictions.float()
    labels = labels.float()
    sensitive_attribute = sensitive_attribute.float()

    def statistical_parity_difference():
        prob_group_1 = predictions[sensitive_attribute == 1].mean()
        prob_group_0 = predictions[sensitive_attribute == 0].mean()
        return abs(prob_group_1 - prob_group_0), prob_group_0, prob_group_1

    def equal_opportunity_difference():
        tpr_group_1 = predictions[(labels == 1) & (sensitive_attribute == 1)].mean()
        tpr_group_0 = predictions[(labels == 1) & (sensitive_attribute == 0)].mean()
        return abs(tpr_group_1 - tpr_group_0), tpr_group_0, tpr_group_1

    def overall_accuracy_equality_difference():
        acc_group_1 = (predictions[sensitive_attribute == 1] == labels[sensitive_attribute == 1]).float().mean()
        acc_group_0 = (predictions[sensitive_attribute == 0] == labels[sensitive_attribute == 0]).float().mean()
        return abs(acc_group_1 - acc_group_0), acc_group_0, acc_group_1

    def treatment_equality_difference():
        fn_group_1 = ((predictions == 0) & (labels == 1) & (sensitive_attribute == 1)).sum()
        fp_group_1 = ((predictions == 1) & (labels == 0) & (sensitive_attribute == 1)).sum()

        fn_group_0 = ((predictions == 0) & (labels == 1) & (sensitive_attribute == 0)).sum()
        fp_group_0 = ((predictions == 1) & (labels == 0) & (sensitive_attribute == 0)).sum()

        ratio_group_1 = fn_group_1 / fp_group_1 if fp_group_1 != 0 else float('inf')
        ratio_group_0 = fn_group_0 / fp_group_0 if fp_group_0 != 0 else float('inf')

        return abs(ratio_group_1 - ratio_group_0), ratio_group_0, ratio_group_1, fn_group_1, fp_group_1, fn_group_0, fp_group_0

    # Calculating each fairness metric
    spd, sp_g0, sp_g1 = statistical_parity_difference()
    eod, eod_g0, eod_g1 = equal_opportunity_difference()
    oaed, oaed_g0, oaed_g1 = overall_accuracy_equality_difference()
    ted, ted_g0, ted_g1, fn_group_1, fp_group_1, fn_group_0, fp_group_0 = treatment_equality_difference()

    return {
        'Statistical Parity Difference': spd,
        'Statistical Parity Group with S=0': sp_g0,
        'Statistical Parity Group S=1': sp_g1,
        'Equal Opportunity Difference': eod,
        'Equal Opportunity Group with S=0': eod_g0,
        'Equal Opportunity Group S=1': eod_g1,
        'Overall Accuracy Equality Difference': oaed,
        'Overall Accuracy Group with S=0': oaed_g0,
        'Overall Accuracy Group S=1': oaed_g1,
        'Treatment Equality Difference': ted,
        'Treatment Equality Group with S=0': ted_g0,
        'Treatment Equality Group S=1': ted_g1
        # 'False Negatives Group 1': fn_group_1,
        # 'False Positives Group 1': fp_group_1,
        # 'False Negatives Group 0': fn_group_0,
        # 'False Positives Group 0': fp_group_0
    }

# ------------------------------------------------------------------
# Model Training
# ------------------------------------------------------------------

# Train the model
def training(model, data, optimizer, epochs=2000, fairness=False, alpha=0, beta=0, gamma=0, delta=0):
    model.to(set_device())
    data.to(set_device())
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        
        if fairness:
            loss = fairness_aware_loss(out[data.train_mask], data, data.x[data.train_mask, -1],
                                       alpha=alpha, beta=beta, gamma=gamma, delta=delta)
            
        else:
            # criterion = torch.nn.CrossEntropyLoss()
            # criterion = torch.nn.BCELoss()
            criterion = torch.nn.NLLLoss()
            loss = criterion(out[data.train_mask], data.y[data.train_mask])

        loss.backward()
        optimizer.step()

        metrics = test(model, data)

        if epoch % 10 == 0:
            print(f'Epoch {epoch} | Loss: {loss.item()} | \n AUC_ROC: {metrics["AUC_ROC"]} | F1 Score: {metrics["F1_Score"]} | SPD: {metrics["parity"]} | EOD: {metrics["equality"]}')

# ------------------------------------------------------------------
# Model Testing
# ------------------------------------------------------------------

# Test the model
def test(model, data, balanced=False):
    # model.to('cpu')
    # data.to('cpu')
    model.to(set_device())
    data.to(set_device())
    
    model.eval()
    with torch.inference_mode():
      out = model(data.x, data.edge_index)

    _, pred = model(data.x, data.edge_index).max(dim=1)
    correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
    accuracy = correct / int(data.test_mask.sum())
    # print(f'Accuracy: {accuracy}')

    # Convert model outputs to binary predictions
    predictions = out.argmax(dim=1)

    fairness_metrics = calculate_fairness(label='GoodCustomer', predictions=predictions, sens_attr='Gender', balanced=balanced)
    fairness_metrics['Accuracy'] = accuracy

    return fairness_metrics

# ------------------------------------------------------------------
# Print Metrics
# ------------------------------------------------------------------

# def print_metrics(metrics):
#     for key, value in metrics.items():
#         print(f"\n{key} : {value:.5f}")

def print_metrics(metrics):
    count = -1

    for key, value in metrics.items():
        count += 1
        if count == 3:
            print(f"\n\n{key} : {value:.5f}")
            count = 0
        else:
            print(f"{key} : {value:.5f}")

/home/mahadik/miniconda3/envs/fairgnn_env/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def test(model, data, val=True, balanced=False):
    model.to(set_device())
    data.to(set_device())
    
    if val==True:
      mask = data.val_mask
    else:
      mask = data.test_mask

    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        predictions = out.argmax(dim=1)

    # Compute accuracy
    correct = int(predictions[mask].eq(data.y[mask]).sum().item())
    accuracy = correct / int(mask.sum())
    
    # Extract the predictions and the true labels
    y_true = data.y[mask].cpu().numpy()
    y_pred = predictions[mask].cpu().numpy()
    
    # Compute F1 score
    f1 = f1_score(y_true, y_pred, average='binary')

    # Compute AUC-ROC score
    y_probs = out[mask][:, 1].cpu().numpy() 
    auc_roc = roc_auc_score(y_true, y_probs)
    
    fairness_metrics = fair_metric('NoDefaultNextMonth', predictions, 'Age')
    fairness_metrics['Accuracy'] = accuracy
    fairness_metrics['F1_Score'] = f1
    fairness_metrics['AUC_ROC'] = auc_roc

    return fairness_metrics

def fair_metric(labels, pred, sens):
	
	labels = user_labels[labels].values
	sens = user_labels[sens].values
	
	idx_s0 = sens==0
	idx_s1 = sens==1

	idx_s0_y1 = np.bitwise_and(idx_s0, labels==1)
	idx_s1_y1 = np.bitwise_and(idx_s1, labels==1)

	parity = abs(sum(pred[idx_s0])/sum(idx_s0)-sum(pred[idx_s1])/sum(idx_s1))
	equality = abs(sum(pred[idx_s0_y1])/sum(idx_s0_y1)-sum(pred[idx_s1_y1])/sum(idx_s1_y1))
    
	return {"parity": parity.item(), "equality": equality.item()}

In [ ]:
sens_attribute_tensor = torch.tensor(user_labels['Age'].values, dtype=torch.long)
sens_attribute_tensor = sens_attribute_tensor.to(set_device())

# GCN Model

In [3]:
class GCN(nn.Module):
	def __init__(self, nfeat, nhid=128, nclass=2, dropout=0):
		super(GCN, self).__init__()
		self.body = GCN_Body(nfeat,nhid,dropout)
		self.fc = nn.Linear(nhid, nclass)

		for m in self.modules():
			self.weights_init(m)

	def weights_init(self, m):
		if isinstance(m, nn.Linear):
			torch.nn.init.xavier_uniform_(m.weight.data)
			if m.bias is not None:
				m.bias.data.fill_(0.0)

	def forward(self, x, edge_index):
		x = self.body(x, edge_index)
		x = self.fc(x)
		return F.log_softmax(x, dim=1)
		# return x

In [4]:
class GCN_Body(nn.Module):
	def __init__(self, nfeat, nhid, dropout):
		super(GCN_Body, self).__init__()
		self.gc1 = GCNConv(nfeat, nhid)

	def forward(self, x, edge_index):
		x = self.gc1(x, edge_index)
		return x

In [9]:
gcn_model = GCN(data.num_node_features, nhid=128, nclass=2)
optimizer_gcn_model = torch.optim.Adam(gcn_model.parameters(), lr=1e-4, weight_decay=1e-5)

In [10]:
training(model=gcn_model, 
         data=data, 
         optimizer=optimizer_gcn_model, 
         fairness=False,  
         epochs=2000)

Epoch 0 | Loss: 1100.4736328125 | 
 AUC_ROC: 0.49777059469099383 | F1 Score: 0.002089864158829676 | SPD: 0.00031824037432670593 | EOD: 0.0007496429025195539
Epoch 10 | Loss: 962.0084228515625 | 
 AUC_ROC: 0.5017080218121956 | F1 Score: 0.0041753653444676405 | SPD: 0.00014500808902084827 | EOD: 0.0010344861075282097
Epoch 20 | Loss: 824.3636474609375 | 
 AUC_ROC: 0.5068061938404554 | F1 Score: 0.009575353871773521 | SPD: 0.00036957627162337303 | EOD: 0.0022535158786922693
Epoch 30 | Loss: 687.7144775390625 | 
 AUC_ROC: 0.5131090952740243 | F1 Score: 0.021087450899317756 | SPD: 0.0010256925597786903 | EOD: 0.003161635249853134
Epoch 40 | Loss: 553.314697265625 | 
 AUC_ROC: 0.5248508985527128 | F1 Score: 0.06673366834170855 | SPD: 0.005320094525814056 | EOD: 0.004543326795101166
Epoch 50 | Loss: 427.0456848144531 | 
 AUC_ROC: 0.5394100246930517 | F1 Score: 0.230727371666969 | SPD: 0.04201469570398331 | EOD: 0.04759699106216431
Epoch 60 | Loss: 315.5310974121094 | 
 AUC_ROC: 0.553928853144

In [11]:
print("Here are the values for the GCN model")

metrics_gcn_model = test(gcn_model, data)

print_metrics(metrics_gcn_model)

Here are the values for the GCN model
parity : 0.04121
equality : 0.07552
Accuracy : 0.54733


F1_Score : 0.63806
AUC_ROC : 0.65361


# FAME

In [12]:
class FairnessAwareMessagePassingLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(FairnessAwareMessagePassingLayer, self).__init__(aggr='mean')  
        self.lin = nn.Linear(in_channels, out_channels)
        self.sensitive_attr = sens_attribute_tensor
        self.bias_correction = nn.Parameter(torch.rand(1))

    def forward(self, x, edge_index):        
        # Add self-loops 
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        x = self.lin(x)

        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)
    
    def message(self, x_j, edge_index, size):
        row, col = edge_index
        deg = degree(row, size[0], dtype=x_j.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        
        group_difference = self.sensitive_attr[row] - self.sensitive_attr[col]
        
        # Adjust messages based on statistical parity
        fairness_adjustment = (1 + self.bias_correction * group_difference.view(-1, 1))

        return fairness_adjustment * norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        return aggr_out

In [13]:
class FairMP_GCN(torch.nn.Module):
    def __init__(self, data, layers=1, hidden=128, dropout=0):
        super(FairMP_GCN, self).__init__()
        self.conv1 = FairnessAwareMessagePassingLayer(data.num_node_features, hidden)
        self.convs = torch.nn.ModuleList()
        
        for i in range(layers - 1):
            self.convs.append(FairnessAwareMessagePassingLayer(hidden, hidden))
        
        # self.conv2 = FairnessAwareMessagePassingLayer(hidden, 2)
        self.fc = nn.Linear(hidden, 2)
        self.dropout = dropout

    def forward(self, x, edge_index, *args, **kwargs):
        x = F.relu(self.conv1(x, edge_index))
        # x = F.dropout(x, p=self.dropout, training=self.training)

        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
            x = F.dropout(x, p=self.dropout, training=self.training)

        # x = self.conv2(x, edge_index)
        x = self.fc(x)
        
        return F.log_softmax(x, dim=1)

In [14]:
Fair_gcn_model = FairMP_GCN(data, hidden=128)
optimizer_Fair_gcn_model = torch.optim.Adam(Fair_gcn_model.parameters(), lr=1e-4, weight_decay=1e-5)

In [15]:
training(model=Fair_gcn_model, 
         data=data, 
         optimizer=optimizer_Fair_gcn_model, 
         fairness=False,  
         epochs=2000)

Epoch 0 | Loss: 3.5742223262786865 | 
 AUC_ROC: 0.5562681768296865 | F1 Score: 0.14562920268972143 | SPD: 0.02087133750319481 | EOD: 0.0222148597240448
Epoch 10 | Loss: 2.1182377338409424 | 
 AUC_ROC: 0.5492175389258522 | F1 Score: 0.31204943357363546 | SPD: 0.005294740200042725 | EOD: 0.001901775598526001
Epoch 20 | Loss: 1.0017176866531372 | 
 AUC_ROC: 0.4437280506207559 | F1 Score: 0.5768833849329206 | SPD: 0.10441845655441284 | EOD: 0.10122531652450562
Epoch 30 | Loss: 0.9074974656105042 | 
 AUC_ROC: 0.46729293847314635 | F1 Score: 0.7954734219269104 | SPD: 0.13248145580291748 | EOD: 0.1377316117286682
Epoch 40 | Loss: 0.9247790575027466 | 
 AUC_ROC: 0.4691133651142053 | F1 Score: 0.8026356429527437 | SPD: 0.1296091079711914 | EOD: 0.13124221563339233
Epoch 50 | Loss: 0.8750569820404053 | 
 AUC_ROC: 0.4691925029151519 | F1 Score: 0.795556017028346 | SPD: 0.1252347230911255 | EOD: 0.13051921129226685
Epoch 60 | Loss: 0.8287934064865112 | 
 AUC_ROC: 0.46459668015638933 | F1 Score: 0.

In [16]:
print("Here are the values for the GCN model")

metrics_Fair_gcn_model = test(Fair_gcn_model, data)

print_metrics(metrics_Fair_gcn_model)

Here are the values for the GCN model
parity : 0.01853
equality : 0.00912
Accuracy : 0.80467


F1_Score : 0.89004
AUC_ROC : 0.65599


# A-FAME

In [6]:
import torch
from torch_geometric.nn import MessagePassing
from torch.nn import Linear, Parameter
from torch_geometric.utils import add_self_loops, softmax
import torch.nn.functional as F

class Attention_FairMessagePassing(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(Attention_FairMessagePassing, self).__init__(aggr='add') 
        self.lin = Linear(in_channels, out_channels) 
        self.att = Linear(2 * out_channels, 1) 
        
        self.sensitive_attr = sens_attribute_tensor 
        self.bias_correction = Parameter(torch.rand(1))  

    def forward(self, x, edge_index):
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        x = self.lin(x)

        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, edge_index, x_i, x_j, size_i):
        x_cat = torch.cat([x_i, x_j], dim=-1)  
        alpha = self.att(x_cat)

        row, col = edge_index
        group_difference = self.sensitive_attr[row] - self.sensitive_attr[col]

        fairness_adjustment = self.bias_correction * group_difference.view(-1, 1)
        alpha = alpha + fairness_adjustment

        alpha = softmax(alpha, edge_index[0], num_nodes=size_i)

        return alpha * x_j

    def update(self, aggr_out):
        return aggr_out
    
# GCN class that takes in the data as an input for dimensions of the convolutions
class Fair_Attention_MP_GCN(torch.nn.Module):
    def __init__(self, data, layers=1, hidden=128, dropout=0):
        super(Fair_Attention_MP_GCN, self).__init__()
        self.conv1 = Attention_FairMessagePassing(data.num_node_features, hidden)
        self.convs = torch.nn.ModuleList()
        
        for i in range(layers - 1):
            self.convs.append(Attention_FairMessagePassing(hidden, hidden))
        
        # self.conv2 = Attention_FairMessagePassing(hidden, 2)
        self.fc = Linear(hidden, 2)
        self.dropout = dropout

    def forward(self, x, edge_index, *args, **kwargs):
        x = F.relu(self.conv1(x, edge_index))
        # x = F.dropout(x, p=self.dropout, training=self.training)

        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
            # x = F.dropout(x, p=self.dropout, training=self.training)

        # x = self.conv2(x, edge_index)
        x = self.fc(x)
        
        return F.log_softmax(x, dim=1)

In [7]:
Fair_gat_model = Fair_Attention_MP_GCN(data, hidden=128)
optimizer_Fair_gat_model = torch.optim.Adam(Fair_gat_model.parameters(), lr=1e-4, weight_decay=1e-5)

training(model=Fair_gat_model, 
         data=data, 
         optimizer=optimizer_Fair_gat_model, 
         fairness=False,  
         epochs=2000)

Epoch 0 | Loss: 135.48150634765625 | 
 AUC_ROC: 0.5170123979696823 | F1 Score: 0.5994995390491242 | SPD: 0.0016413331031799316 | EOD: 0.010821223258972168
Epoch 10 | Loss: 96.00503540039062 | 
 AUC_ROC: 0.534057205569655 | F1 Score: 0.6334355828220859 | SPD: 0.04429858922958374 | EOD: 0.03798985481262207
Epoch 20 | Loss: 61.07234191894531 | 
 AUC_ROC: 0.5528203237533438 | F1 Score: 0.6681465038845726 | SPD: 0.019841313362121582 | EOD: 0.010318934917449951
Epoch 30 | Loss: 28.793062210083008 | 
 AUC_ROC: 0.5645171136566294 | F1 Score: 0.7072477498815727 | SPD: 0.016036570072174072 | EOD: 0.010111033916473389
Epoch 40 | Loss: 4.6422834396362305 | 
 AUC_ROC: 0.5298902531037795 | F1 Score: 0.8819665978607618 | SPD: 0.0020407438278198242 | EOD: 0.000422060489654541
Epoch 50 | Loss: 7.621397495269775 | 
 AUC_ROC: 0.5290324267782427 | F1 Score: 0.8837426463722103 | SPD: 0.0017423033714294434 | EOD: 0.0032573938369750977
Epoch 60 | Loss: 6.789791584014893 | 
 AUC_ROC: 0.5354231257287879 | F1 S

In [8]:
print("Here are the values for the FairGAT model")

metrics_Fair_gat_model = test(Fair_gat_model, data)

print_metrics(metrics_Fair_gat_model)

Here are the values for the FairGAT model
parity : 0.00312
equality : 0.00121
Accuracy : 0.81417


F1_Score : 0.89203
AUC_ROC : 0.73297


# GAT Model

In [9]:
class GAT(nn.Module):
	def __init__(self, nfeat, nhid=128, nclass=2, dropout=0):
		super(GAT, self).__init__()
		self.body = GAT_Body(nfeat,nhid,dropout)
		self.fc = nn.Linear(nhid, nclass)

		for m in self.modules():
			self.weights_init(m)

	def weights_init(self, m):
		if isinstance(m, nn.Linear):
			torch.nn.init.xavier_uniform_(m.weight.data)
			if m.bias is not None:
				m.bias.data.fill_(0.0)

	def forward(self, x, edge_index):
		x = self.body(x, edge_index)
		x = self.fc(x)
		return F.log_softmax(x, dim=1)
		# return x

In [15]:
from torch_geometric.nn import GATConv

class GAT_Body(nn.Module):
	def __init__(self, nfeat, nhid, dropout):
		super(GAT_Body, self).__init__()
		self.gc1 = GATConv(nfeat, nhid)

	def forward(self, x, edge_index):
		x = self.gc1(x, edge_index)
		return x

In [17]:
gat_model = GAT(data.num_node_features, nhid=128)
optimizer_gat_model = torch.optim.Adam(gat_model.parameters(), lr=1e-4, weight_decay=1e-5)

training(model=gat_model, 
         data=data, 
         optimizer=optimizer_gat_model, 
         fairness=False,  
         epochs=2000)

Epoch 0 | Loss: 236.0264129638672 | 
 AUC_ROC: 0.5389471157143837 | F1 Score: 0.5140420526393177 | SPD: 0.1579587310552597 | EOD: 0.1714525818824768
Epoch 10 | Loss: 179.31912231445312 | 
 AUC_ROC: 0.5482942074216339 | F1 Score: 0.5747922437673131 | SPD: 0.15254884958267212 | EOD: 0.1602594256401062
Epoch 20 | Loss: 132.0421600341797 | 
 AUC_ROC: 0.5625078880581659 | F1 Score: 0.6686686686686687 | SPD: 0.17194166779518127 | EOD: 0.17704641819000244
Epoch 30 | Loss: 95.77946472167969 | 
 AUC_ROC: 0.5740366280266136 | F1 Score: 0.7351401651738978 | SPD: 0.16422495245933533 | EOD: 0.158677339553833
Epoch 40 | Loss: 69.81714630126953 | 
 AUC_ROC: 0.588770920502092 | F1 Score: 0.7883981542518128 | SPD: 0.12261301279067993 | EOD: 0.11320215463638306
Epoch 50 | Loss: 56.08202362060547 | 
 AUC_ROC: 0.601948950545305 | F1 Score: 0.8273703974131636 | SPD: 0.11642575263977051 | EOD: 0.10564464330673218
Epoch 60 | Loss: 47.46956253051758 | 
 AUC_ROC: 0.6085918958776322 | F1 Score: 0.84687246141348

In [18]:
print("Here are the values for the GAT model")

metrics_gat_model = test(gat_model, data)

print_metrics(metrics_gat_model)

Here are the values for the GAT model
parity : 0.02615
equality : 0.01580
Accuracy : 0.81617


F1_Score : 0.89358
AUC_ROC : 0.73912


# Compare results

In [ ]:
Values for the FairGAT model - same model specifications as the one from Carlos,
but with one A-FAME instead of one GCNConv layer

parity : 0.00706
equality : 0.00207
Accuracy : 0.68000
F1_Score : 0.80952
AUC_ROC : 0.67496